# Timing analysis of chain CZ gate - Tensorflow implementation

In [ ]:
# Import packages
import numpy as np
import time
import tensorflow as tf
from joblib import Parallel, delayed
import csv
import os

# Set data type
data_type = tf.complex64

## Time-dependent Shrödinger equation solvers

In [ ]:
# Spectral method solver
def unit_evol(H, psi, tau):

    U = tf.linalg.expm(-1j*H*tau)
    psi = tf.linalg.matmul(U,psi) 
    
    return psi

# Crank-Nicolson solver
def crank_nicolson(H, psi, tau, n_iter):
    
    t = np.linspace(0, tau, n_iter)
    dt = t[1] - t[0]
    
    A = tf.linalg.eye(H.shape[0], dtype=data_type) + 1j * H * dt/2
    B = tf.linalg.eye(H.shape[0], dtype=data_type) - 1j * H * dt/2

    b = tf.linalg.matmul(B,psi) 
    
    for index, step in enumerate(t):
        psi = tf.linalg.solve(A, b)
        b = tf.linalg.matmul(B,psi) 
        
    return psi

# Crank-Nicolson LU optimized solver
def crank_nicolson_LU(H, psi, tau, n_iter):
    
    t = np.linspace(0, tau, n_iter)
    dt = t[1] - t[0]
    
    A = tf.linalg.eye(H.shape[0], dtype=data_type) + 1j * H * dt/2
    B = tf.linalg.eye(H.shape[0], dtype=data_type) - 1j * H * dt/2

    # Initial conditions
    b = tf.linalg.matmul(B,psi) 
    
    LU, p = tf.linalg.lu(A)
    
    for index, step in enumerate(t):
       
        psi = tf.linalg.lu_solve(LU,p,b)
        b = tf.linalg.matmul(B,psi) 
    
    return psi  

## Chain CZ-gate implementation

In [ ]:
# Tensor operation
def tensor(a,b):
    
    a_shape = [a.shape[0],a.shape[1]]
    b_shape = [b.shape[0],b.shape[1]]
    
    return tf.reshape(tf.reshape(a,[a_shape[0],1,a_shape[1],1])*tf.reshape(b,[1,b_shape[0],1,b_shape[1]]),[a_shape[0]*b_shape[0],a_shape[1]*b_shape[1]])

In [ ]:
# Computational base vector initialization
def basis(dim, state):
    vect = np.zeros((dim,1))
    vect[state] = 1
    vect = tf.constant(vect, dtype=data_type, shape=(dim,1))
    return vect

In [ ]:
# Optimal phase between two pulses
def exp_xi(Delta,Omega,tau):
    
    y = Delta/Omega
    s = Omega * tau
    
    a = np.sqrt(y**2+1)
    b = s*a/2

    return (a*np.cos(b) + 1j*y*np.sin(b)) / (-a*np.cos(b) + 1j*y*np.sin(b))

In [ ]:
# Definition of the Hamiltonian for a two-qubit CZ gate
def hamiltonian(Omega,Delta):
    
    psi00 = tensor(basis(3,0),basis(3,0))
    psi01 = tensor(basis(3,0),basis(3,1)) 
    psi0r = tensor(basis(3,0),basis(3,2))
    psi10 = tensor(basis(3,1),basis(3,0))
    psi11 = tensor(basis(3,1),basis(3,1)) 
    psi1r = tensor(basis(3,1),basis(3,2))
    psir0 = tensor(basis(3,2),basis(3,0))
    psir1 = tensor(basis(3,2),basis(3,1))
    psirr = tensor(basis(3,2),basis(3,2))    

    H0  = 0 * tensor( tf.linalg.adjoint(psi00),psi00)
    
    H01 = 1/2 * ( Omega * tensor( tf.linalg.adjoint(psi01),psi0r) + 
             np.conj(Omega) * tensor( tf.linalg.adjoint(psi0r),psi01) ) - Delta * tensor( tf.linalg.adjoint(psi0r),psi0r)
    
    H10 = 1/2 * ( Omega * tensor( tf.linalg.adjoint(psi10),psir0) + 
             np.conj(Omega) * tensor( tf.linalg.adjoint(psir0),psi10) ) - Delta * tensor( tf.linalg.adjoint(psir0),psir0)

    H2  = 1/2 * ( Omega * ( tensor( tf.linalg.adjoint(psi11),psir1) + tensor( tf.linalg.adjoint(psi11),psi1r) ) 
            + np.conj(Omega) * ( tensor( tf.linalg.adjoint(psir1),psi11) + tensor( tf.linalg.adjoint(psi1r),psi11) ) 
            ) - Delta/2 * ( tensor( tf.linalg.adjoint(psir1),psir1) + tensor( tf.linalg.adjoint(psir1),psi1r) 
                          + tensor( tf.linalg.adjoint(psi1r),psir1) + tensor( tf.linalg.adjoint(psi1r),psi1r))

    H = H0 + H01 + H10 + H2
    
    return H

In [ ]:
# Chain state initialization
def chain_init(N,state_first,state_last):
    
    psi = basis(3,state_first) 
    
    for i in range(N-2):
        psi = tensor(psi,basis(3,0))
        
    psi = tensor(psi,basis(3,state_last))
    
    return psi

In [ ]:
# Chain CZ_gate implementation
def chain_CZ_gate_time(psi,Omega,Delta,tau,method='spectral_evol',niter=100):
    
    start = time.time()
    
    N = int(np.log10(psi.shape[0])/(np.log10(3))) # Number of qubits in the chain
    I = tf.linalg.eye(3, dtype=data_type) # Define identity matrix
    Hp1 = hamiltonian(Omega,Delta) # First pulse hamiltonian
    Hp2 = hamiltonian(Omega * exp_xi(Delta,Omega,tau), Delta) # Second pulse hamiltonian
    
    for i in range(N-1):
        
        mat_list1 = [I]*(N-1)
        mat_list2 = [I]*(N-1)
        mat_list1[i] = Hp1
        mat_list2[i] = Hp2
        
        H1 = mat_list1[0]
        H2 = mat_list2[0]
        
        for j in range(N-2):
            H1 = tensor(H1, mat_list1[j+1] )
            H2 = tensor(H2, mat_list2[j+1] )
        
        if method == 'spectral_evol':
            psi = unit_evol(H1, psi, tau)   
            psi = unit_evol(H2, psi, tau) 
        
        elif method == 'cn':
            psi = crank_nicolson(H1, psi, tau, niter)
            psi = crank_nicolson(H2, psi, tau, niter)
        
        elif method == 'cn_LU':
            psi = crank_nicolson_LU(H1, psi, tau, niter)
            psi = crank_nicolson_LU(H2, psi, tau, niter)
            
        else: 
            print("ERROR: no valid input method!")
            return None
    
    end = time.time()
        
    return end-start

## Timing analysis as a function of the number of qubit N

### CPU

In [ ]:
def chain_CZ_gate_execution_CPU(N, method, niter=100, ntimes=10):

    Omega   = 1
    frac_DO = 0.377371
    prod_Ot = 4.29268
    Delta = frac_DO * Omega
    tau = prod_Ot / Omega

    filename = "tensorflow_CPU_"+str(method)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)
    
    print("Method: ", method)
    for i in range(len(N)):
        print("N: ", N[i])
        
        state_first = 1
        state_last = 1
        psi_init = chain_init(N[i],state_first,state_last)
        
        res = Parallel(n_jobs=-1, verbose=11)(delayed(chain_CZ_gate_time)(psi_init,Omega,Delta,tau,method,niter) for t in range(ntimes))


        with open(filename, "a") as f:
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res), np.std(res)/np.sqrt(ntimes)))

In [ ]:
# Spectral method
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution_CPU(N, 'spectral_evol', ntimes=10)

In [ ]:
# Crank-Nicolson method
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution_CPU(N, 'cn', 100, ntimes=10)

In [ ]:
# Crank-Nicolson method with LU decomposition
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution_CPU(N, 'cn_LU', 100, ntimes=10)

### GPU

In [ ]:
def chain_CZ_gate_execution_GPU(N, method, niter=100, ntimes=10):

    Omega   = 1
    frac_DO = 0.377371
    prod_Ot = 4.29268
    Delta = frac_DO * Omega
    tau = prod_Ot / Omega

    filename = "tensorflow_GPU_"+str(method)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)
    
    print("Method: ", method)
    for i in range(len(N)):
        
        state_first = 1
        state_last = 1
        psi_init = chain_init(N[i],state_first,state_last)
        
        res = []
        
        for j in range(ntimes):
            print("N: ", N[i], "ntimes: ", j)
            res.append(chain_CZ_gate_time(psi_init,Omega,Delta,tau,method,niter))


        with open(filename, "a") as f:
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res), np.std(res)/np.sqrt(ntimes)))

In [ ]:
# Spectral method
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution_GPU(N, 'spectral_evol', ntimes=10)

In [ ]:
# Crank-Nicolson method
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution_GPU(N, 'cn', 100, ntimes=10)

In [ ]:
# Crank-Nicolson method with LU decomposition
N = [2, 3, 4, 5, 6, 7]
chain_CZ_gate_execution_GPU(N, 'cn_LU', 100, ntimes=10)